Credit to @wenhuchen -  https://github.com/wenhuchen/Program-of-Thoughts

In [6]:
import json
from time import sleep
from tqdm import tqdm
import os
import openai
from datetime import datetime
from tool import finqa_equal, parse_api_result, safe_execute
from eval_tatqa.tatqa_utils import extract_one_num_from_str
from typing import Dict, Any
import argparse
from collections import Counter

In [7]:
def create_reader_request_processed(example: Dict[str, Any]):
    prompt = 'Read the following text and table, and then answer the last question in a series of questions:\n'
    if example['golden_text']:
        prompt += example['golden_text'].strip() + '\n'
    if example['golden_table']:
        prompt += example['golden_table'].strip() + '\n'
    #prompt += '\n'
    prompt += 'Questions: '
    prompt += " ".join(example['questions'][:-1])
    prompt += '\n'
    prompt += f'Question: {example["questions"][-1]}'
    return prompt

In [8]:
prompt_msg_4shot = [{"role":"system","content":"You are an AI assistant that helps people find information. You will be given a context a question and will be asked to provide an answer in the form of a program.  Ensure the program has a variable called 'ans' which provides the answer to the last question."},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\n- | shares available for awards | shares subject to outstanding awards\n2009 global incentive plan | 2322450 | 2530454\n2004 stock incentive plan | - | 5923147\nQuestions: how many shares are subject to outstanding awards is under the 2009 global incentive plan? what about under the 2004 stock incentive plan? how many total shares are subject to outstanding awards? what about under the 2004 stock incentive plan?\nQuestion: what proportion does this represent?"},{"role":"assistant","content":"#Python\nshares_subject_to_outstanding_awards_2009_global_incentive_plan = 2530454\nshares_subject_to_outstanding_awards_2004_stock_incentive_plan = 5923147\ntotal_shares_subject_to_outstanding_awards = shares_subject_to_outstanding_awards_2009_global_incentive_plan + shares_subject_to_outstanding_awards_2004_stock_incentive_plan\nproportion = shares_subject_to_outstanding_awards_2009_global_incentive_plan / total_shares_subject_to_outstanding_awards\nans = proportion"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\ncompensation expense the company recorded $ 43 million , $ 34 million , and $ 44 million of expense related to stock awards for the years ended december 31 , 2015 , 2014 , and 2013 , respectively . \nQuestions: what is the compensation expense the company recorded in 2015? what about in 2014? what is the total compensation expense the company recorded in 2015 and 2014? what is the total expenses including 2013?\nQuestion: what is the average for three years?"},{"role":"assistant","content":"#Python\ncompensation_expense_2015 = 43\ncompensation_expense_2014 = 34\ncompensation_expense_2013 = 44\ntotal_compensation_expense = compensation_expense_2015 + compensation_expense_2014 + compensation_expense_2013\naverage_for_three_years = total_compensation_expense / 3\nans = average_for_three_years\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nthe net loss on disposal of those assets was $ 344000 for 2005 and $ 43000 for 2004 . \nQuestions: what was the net loss on disposal of assets in 2005? what was the value in 2004? what was the change in value?\nQuestion: what was the percent change?"},{"role":"assistant","content":"#Python\nnet_loss_on_disposal_of_assets_2005 = 344000\nnet_loss_on_disposal_of_assets_2004 = 43000\nnet_change_in_value = net_loss_on_disposal_of_assets_2005 - net_loss_on_disposal_of_assets_2004\npercent_change = net_change_in_value / net_loss_on_disposal_of_assets_2004\nans = percent_change\n"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:\nlocation | operations conducted | approximatesquare feet | leaseexpirationdates\ndublin ireland | global supply chain distribution and administration offices | 160000 | owned\nathlone ireland | commercial research and development manufacturing | 80000 | owned\nbogart georgia | commercial research and development manufacturing | 70000 | owned\nsmithfield rhode island | commercial research and development manufacturing | 67000 | owned\nQuestions: what is the square feet of the owned global supply chain distribution and administration offices? what is the square feet of the owned commercial research and development manufacturing? what is the sum of those values? what is the total sum including square feet of commercial research and development manufacturing in bogart, georgia? what is the total sum including square feet of commercial research and development manufacturing in smithfield, rhode island?\nQuestion: what is the total sum of square feet owned?"},{"role":"assistant","content":"#Python\nsquare_feet_owned = [160000, 80000, 70000, 67000]\ntotal_square_feet_owned = sum(square_feet_owned)\nans = total_square_feet_owned"},{"role":"user","content":"Read the following text and table, and then answer the last question in a series of questions:  \nentergy new orleans estimates that approximately 141000 electric customers and 93000 gas customers have returned since hurricane katrina and are taking service as of december 31 , 2008 , compared to approximately 132000 electric customers and 86000 gas customers as of december 31 , 2007 .  \nQuestions: what was the number of gas customers in 2008? and what was it in 2007? what was, then, the change in that number over the year?  \nQuestion: and how much does this change represent in relation to the 2007 number of customers, in percentage?"},{"role":"assistant","content":"#Python\ngas_customers_2008 = 93000\ngas_customers_2007 = 86000\nchange_in_gas_customers = gas_customers_2008 - gas_customers_2007\npercent_change = (change_in_gas_customers / gas_customers_2007) / gas_customers_2007\nans = percent_change"}]

In [9]:
import os
import openai
from configparser import ConfigParser

parser=ConfigParser()
_=parser.read('../../config.cfg')
openai.api_type = "azure"
openai.api_base = parser.get('openai_api','api_ep')
openai.api_version = "2023-03-15-preview"
openai.api_key = parser.get('openai_api','api_key')
model =  'gpt-4' #parser.get('openai_api','api_model')

os.environ["OPENAI_API_KEY"]  = openai.api_key
os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_VERSION"] = openai.api_version
os.environ["OPENAI_API_BASE"] = openai.api_base


In [10]:
with open('data/convfinqa_test_turn.json') as f:
        convfinqa_dev = json.load(f)
    
now = datetime.now()
dt_string = now.strftime("%m_%d_%H_%M")

correct, wrong = 0, 0

In [80]:
filename = f'outputs/convfinqa_direct_{model}_{dt_string}.jsonl'
writer = open(filename, 'w')

for example in tqdm(convfinqa_dev):
    full_prompt = prompt_msg_4shot + [{"role":"user","content":create_reader_request_processed(example)}]
    # greedy decoding
    got_result = False
    while not got_result:
        try:
            result = openai.ChatCompletion.create(
                engine=model,
                messages=full_prompt,
                max_tokens=256,
                temperature=0.0,
                top_p=1,
                n=1,
                stop=['\n\n']
            )
            got_result = True
        except Exception:
            sleep(3)
    result_counter = Counter()
    codes = result['choices'][0]['message']['content'],

    for r in codes:
        try :
            exec(r)#extract_one_num_from_str(r)
        except:
            ans=r
        if not ans:
            if 'yes' in r.lower() or 'true' in r.lower():
                ans = 'yes'
            elif 'no' in r.lower() or 'false' in r.lower():
                ans = 'no'
        if ans is not None:
            if type(ans) in [dict]:
                result_counter.update(list(ans.values()))
            elif type(ans) in [list, tuple]:
                result_counter.update([float(ans[0])])
            elif type(ans) in [str]:
                result_counter.update([ans])
            else:
                try:
                    result_counter.update([float(ans)])
                except Exception:
                    continue

    if len(result_counter) > 0:
        prediction = result_counter.most_common(1)[0][0]        
    else:
        prediction = None

    if prediction is None:
        wrong += 1
    elif finqa_equal(prediction, example['answer'], True, True):
        correct += 1
    else:
        wrong += 1

    example.update({'generated': codes, 'executed': prediction})
    writer.write(json.dumps(example) + '\n')
    print('accuracy: ', correct / (correct + wrong))
    
writer.close()


  0%|▏                                                                                 | 1/421 [00:06<42:09,  6.02s/it]

accuracy:  1.0


  0%|▍                                                                                 | 2/421 [00:10<37:06,  5.31s/it]

accuracy:  0.5


  1%|▌                                                                                 | 3/421 [00:14<33:03,  4.75s/it]

...

accuracy:  0.630071599045346


100%|███████████████████████████████████████████████████████████████████████████████▊| 420/421 [30:03<00:05,  5.66s/it]

accuracy:  0.6309523809523809


100%|████████████████████████████████████████████████████████████████████████████████| 421/421 [30:07<00:00,  4.29s/it]

accuracy:  0.6318289786223278


In [81]:
print('accuracy: ', correct / (correct + wrong))


accuracy:  0.6318289786223278
